In [2]:
import numpy as np
# import itertools
from utils.pass_quality import pass_quality, approx_pq
from utils.data import load_data
import matplotlib.pylab as plt

Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, CuDNN 4007)


In [3]:
%matplotlib inline

## Fast pass_quality objective

In [17]:
data = load_data()

X, y = data.drop(['ref_pass'], axis=1), data['ref_pass']

y_true, y_pred = y.values, data['pass.1'].values

y_pass, y_sig = y_true, y_pred

In [18]:
pass_quality(y_true, y_pred)

r= 6416 err= 1602


0.800199551010227

In [19]:
approx_pq(y_true, y_pred)

r= 6414 err= 1605


0.799850355405911

In [20]:
diffs = np.concatenate([[0], np.diff(y_true + y_pred)])[:10000]

In [44]:
def approx_pq_debug(y_pass, y_sig):
    """
    Good approximation for pass_quality function,
    it may be implemented in theano
    -------------
    Arguments:
        y_pass:
        y_sig:
    """
    r = 0
    height = 0
    error = 0
    current_error = 0
    diffs = np.concatenate([[0], np.diff(y_pass + y_sig)])
    flag = 0
    
    for s, s_abs in zip(diffs, np.abs(diffs)):
        
        if height == 2 and flag == 0:
            flag = 1
            current_error = 2    
        
        height = np.max([0, height + s])
        current_error += s_abs
        
        if height == 0:
            error += (current_error) * (current_error != 4) / 3.
            r += (current_error == 4)
            current_error = 0
    
    print 'r=', r, 'err=', error
    epsilon = 0.0001
    return 1 - (float(r) + epsilon) / (r + error + epsilon)

In [45]:
n_start = 3500
n_finish = n_start + 150000

print 1 - pass_quality(y_true[n_start:n_finish], y_pred[n_start:n_finish])
print approx_pq_debug(y_true[n_start:n_finish], y_pred[n_start:n_finish])

r= 6221 err= 1600
0.204577419767
r= 6219 err= 1464.0
0.190550563705


In [46]:
approx_pq_debug(y_true[n_start:], y_pred[n_start:])

r= 6219 err= 1464.0


0.19055056370492673

In [ ]:
v

# almost the same as true pass_quality metrics

In [12]:
#plt.plot(np.linspace(0, len(pqs), num=len(pqs)), pqs)
plt.plot(np.linspace(0, len(approx_pq), ), approx_pqs, c='r')

NameError: name 'approx_pqs' is not defined

# theano implementation

In [8]:
import theano 
from theano import tensor as T
from theano import function
from theano.tensor import concatenate
from theano import scan
from theano.tensor import eq, neq, isclose

In [9]:
# excluding if statement:

In [10]:
T_R = theano.shared(0., 'R')
T_height = theano.shared(0., 'h')
T_error = theano.shared(0., 'err',)
T_current_error = theano.shared(0., 'c_err')
T_diffs = T.vector('diffs')

In [13]:
import theano 
from theano import tensor as T
from theano import function
from theano.tensor import concatenate
from theano import scan
from theano.tensor import eq, neq, isclose

In [14]:
def OneStep(s, s_abs, error, R, height, current_error):
    height += s
    current_error += s_abs
    error += (current_error *  T.switch(isclose(current_error, 4), 0, 1)) / 3. * isclose(height, 0.)
    R += isclose(current_error, 4.) * isclose(height, 0.)
    current_error = current_error - current_error * isclose(height, 0.)
    return error, R, height, current_error

def pq_theano(y_true, y_pred):
    y_true = y_true.astype(dtype='float64')
    y_pred = y_pred.astype(dtype='float64')
    T_diffs  = T.extra_ops.diff(y_true + y_pred)
    
    R = theano.shared(0., 'R')
    height = theano.shared(0., 'h')
    error = theano.shared(0., 'err',)
    current_error = theano.shared(0., 'c_err')
    
    values, updates = scan(fn=OneStep, 
                           sequences = [T.concatenate([[0], T_diffs]), T.abs_(T.concatenate([[0], T_diffs]))],
                           outputs_info = [T.shape_padleft(T.zeros_like(error)), 
                                           T.shape_padleft(T.zeros_like(R)),
                                           T.shape_padleft(T.zeros_like(height)),
                                           T.shape_padleft(T.zeros_like(current_error))])
                            
    epsilon = 0.00000000001
    return 1 - (values[1] + epsilon) / (values[1] + values[0] + epsilon)

In [15]:
y_t, y_p = T.vector(name='y_t', dtype='float32'), T.vector(name='y_p', dtype='float32')

In [16]:
f = function(inputs=[y_t, y_p], outputs=pq_theano(y_t, y_p))

TensorType(float64, vector)


In [18]:
f(y_true.astype('float32'), y_pred.astype('float32'))

array([[ 0.18574331],
       [ 0.18574331],
       [ 0.18574331],
       [ 0.18574331],
       [ 0.18574331],
       [ 0.18574331],
       [ 0.18574331],
       [ 0.18574331],
       [ 0.18574331],
       [ 0.18574331],
       [ 0.18574331],
       [ 0.18574331],
       [ 0.18574331],
       [ 0.18574331],
       [ 0.18574331],
       [ 0.18574331],
       [ 0.18574331],
       [ 0.18574331],
       [ 0.18574331],
       [ 0.18574331],
       [ 0.18574331],
       [ 0.18574331],
       [ 0.18574331],
       [ 0.18574331],
       [ 0.18574331],
       [ 0.18574331],
       [ 0.18574331],
       [ 0.18574331],
       [ 0.18574331],
       [ 0.18574331],
       [ 0.18574331],
       [ 0.18574331],
       [ 0.18574331],
       [ 0.18574331],
       [ 0.18574331],
       [ 0.18574331],
       [ 0.18574331],
       [ 0.18601896],
       [ 0.18601896],
       [ 0.18601896],
       [ 0.18601896],
       [ 0.18601896],
       [ 0.18601896],
       [ 0.18601896],
       [ 0.18601896],
       [ 0